
요청

헤더 -> session에 헤더에 추가해줘야함

Authorization: KakaoAK ${REST_API_KEY}

쿼리 파라미터

-> url에 사용


# 카카오 OpenAPI 연동 (책검색, 페이지 구현)

## #01. 준비과정

### [1] 패키지 참조


In [1]:
import requests
import json
import datetime as dt
import os
from dotenv import load_dotenv

### [2] 접속할 데이터의 URL

In [5]:

urlFmt = 'https://dapi.kakao.com/v3/search/book?query={query}&page={page}&size={size}'



### [3] 요청 변수

In [11]:
query = '파이썬'
page = 1
size = 50
load_dotenv(dotenv_path = '../../../dot_env/.env')

key = os.environ.get("kakao_key")




'797d7693bbdf775bc438033e74ae3402'

## #02. 데이터 요청하기
### [1] 세션 생성

In [6]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Authorization": f"KakaoAK {key}"
})

### [2]데이터 가져오기

마지막 페이지임을 의미하는 'meta > is_end' 라는 값이 'True'로 반환될 때 까지 'page'를 1씩 증가시키면서 반복 수행

In [7]:
page = 1
isEnd = False

# 수집된 데이터를 모아 놓을 빈 리스트
mydata = []

while not isEnd:
    url = urlFmt.format(query=query, page = page, size = size)

    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

        # 에러 발생시 나머지 페이지 중단
        break

        # 에러 발생시 다음 페이지 시도
        # continue
    
    
    r.encoding = "utf-8"
    mydict = json.loads(r.text)

    mydata += mydict['documents']
    print(f'{page} 페이지로부터 {len(mydict['documents'])}건의 데이터를 수집했습니다.')

    page += 1 # 페이지 증가
    isEnd = mydict['meta']['is_end'] # 마지막 페이지인지 확인 // 연동하는 API에 따라 처리방법이 다름

print(mydata)
print('fin :)')

접속에 실패했습니다.
[401 Error] Unauthorized 에러가 발생함
[]
fin :)


## #03. 데이터 활용



In [7]:
with open(f'{query}_책_검색결과.csv','w',encoding='utf-8') as f:
    f.write('제목, 저자, 출판사, 정가, 판매가, 출간일 \n')
    for i, v in enumerate(mydata):
        authors, contents, datetime, isbn, price, publisher, sale_price, status, thumbnail, title, translators, url = v.values()
        authors = "/".join(authors)
        
        # 날짜값이 ISO형식인 경우 python의 datetime객체로 변환하기 위해 fromisoformat() 메서드를 사용한다.
        # 일반적으로 OpenAPI 날짜는 ISO형식이다.
        iso_date = dt.datetime.fromisoformat(datetime)
        f.write(f"{title}, {authors}, {publisher}, {price}, {sale_price}, {iso_date.strftime("%Y-%m-%d")} \n")

print("fin :)")

    

fin :)
